<a href="https://colab.research.google.com/github/LamjahdiMo/Bachelor-Dissertation/blob/main/O(1)-Methods/O(1)_Algorithms_For_LinearEquations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Author: Lamjahdi, Mohamed El Mami
# Date: 07/12/2025
# License: CC BY-NC 4.0

# Generally, prediction during dead time relies on analyzing historical data.
# However, if the system is disturbed and such data is unavailable, then the
# relationships between real-time signals must be studied during the process,
# and accurate predictions must be delivered within milliseconds.
#
# In his thesis, *Dynamic Control of Rolling Force in Mechanical Deep Rolling*,
# the author proposes a method that uses a linear combination of past signals
# to predict future ones. This approach includes a minimally filtered disturbance
# term within the matrix representation, which affects only the constants—
# treated as abstract parameters to be computed during the process.
#
# The number of signals collected plays a pivotal role, as the time required to
# gather them must remain shorter than the total dead time. While generally,
# collecting more signals improves prediction accuracy, it comes at a high cost:
# during the signal acquisition period, the process operates without prediction,
# and thus the subsequent prediction is based on a disturbed or non-predicted phase.
#
# One might assume that this only affects the initial phase of the process
# (e.g., during the onset of deep rolling), after which only one new signal is
# needed to generate a new matrix. However, in reality, each new prediction
# depends on constants computed during the previous cycle, which may introduce
# randomness. When this randomness is compounded with system dead time,
# it can result in control signals that are more akin to disturbances rather
# than precise corrective actions.
#
# This highlights the importance of minimizing randomness in prediction by using
# the smallest feasible number of past signals. In the thesis mentioned above,
# the author uses only 2×2 matrices due to the recursive nature of structured
# programming in TwinCAT, and the need for constant-time solutions.
#
# However, the implementation below leverages the pseudocode from [LAM25],
# which enables direct solutions of 3×3 matrices. This method offers greater
# prediction precision while preserving the O(1) computational complexity,
# aligning with the real-time requirements of industrial control systems.

# A linear combination originally derived from the differential equations of past
# signals is given by:

# x[n] = a * x[n-1] + b * x[n-2] + c * x[n-3]

# Using two additional past signals, we obtain the following system of linear equations:

# x[n]   = a * x[n-1] + b * x[n-2] + c * x[n-3]
# x[n-1] = a * x[n-2] + b * x[n-3] + c * x[n-4]
# x[n-2] = a * x[n-3] + b * x[n-4] + c * x[n-5]

# Assuming each signal is sampled every 1 ms, we require exactly 6 ms of historical data
# to construct a prediction. This allows us to estimate a signal at step n, based on prior values.
# Hence, the 6 ms is only needed at the beginning of the process. Afterward, only 1 ms is required
# to construct a new system of linear equations, since previously computed signals can be
# shifted and reused recursively.

# However, predicting future values several steps ahead using constants derived from this
# linear combination becomes increasingly inaccurate. Both real-world experiments and
# simulations have shown that the prediction loses more than half of its accuracy when the
# predicted step is k steps ahead, where k exceeds the rank of the system matrix. This implies
# that step n+3 is the practical upper bound for reliable prediction using this method.

# Note: This type of step-wise prediction using regularly sampled signals is not intended for
# use during dead time or process stagnation. It is presented here purely for illustration.
# For details on real-time prediction under process latency, please refer to the thesis
# mentioned above.

# By denoting x[n], x[n-1], and x[n-2] as e1, e2, and e3, and treating a, b, and c as the unknowns,
# we form a 3x3 linear system.

# The following code provides constant-time (O(1)) solutions for  linear systems of sizes 2×2 and 3×3
# using direct expressions that avoid iterative methods, matrix inversion, or decomposition.
# It is based on the algebraic techniques and memory-efficient formulations introduced in the following works:

#   • "What If Linear Equations Could Be Solved Instantly?"
#   • "6X-Method for solving 4x4 Linear Equations in O(1)"
#   • "Solutions of 3×3 Systems of Linear Equations Presented on a Golden Plate"

# These studies develop minimal-expression direct solvers for linear systems of dimension 2, 3, and 4.

# --- 2×2 Linear System Solver: A·x = e



def solve2x2(A, e):
    a1 = A[0][0]
    a2 = A[0][1]
    b1 = A[1][0]
    b2 = A[1][1]

    e1 = e[0]
    e2 = e[1]

    denominator = a2 * b1 - b2 * a1
    if denominator == 0:
        return None  # Singular matrix

    numerator_x = a2 * e2 - b2 * e1
    numerator_y = a1 * e2 - b1 * e1

    x = numerator_x / denominator
    y = -numerator_y / denominator

    return [x, y]


# --- 3×3 Linear System Solver: A·x = e
def solve3x3(A, e):
    a1, a2, a3 = A[0]
    b1, b2, b3 = A[1]
    c1, c2, c3 = A[2]

    e1 = e[0]
    e2 = e[1]
    e3 = e[2]

    t1 = a1 * c3 - c1 * a3
    t2 = a2 * c3 - c2 * a3
    t3 = e1 * c3 - e3 * a3

    t4 = b1 * c3 - c1 * b3
    t5 = b2 * c3 - c2 * b3
    t6 = e2 * c3 - e3 * b3

    denominator = t2 * t4 - t5 * t1
    if denominator == 0:
        return None

    numerator_x = t2 * t6 - t5 * t3
    numerator_y = t4 * t3 - t1 * t6

    x = numerator_x / denominator
    y = numerator_y / denominator
    z = (e3 - c1 * x - c2 * y) / c3

    return [x, y, z]


# --- Example Usage
A2 = [
    [1, 2],
    [2, 1],
]
e2 = [5, 4]
print("2x2 solution:", solve2x2(A2, e2))


A3 = [
    [1, 2, 3],
    [2, 1, 1],
    [2, 2, 2]
]
e3 = [14, 7, 12]
print("3x3 solution:", solve3x3(A3, e3))

# Targeting applications that require ultra-low latency and constant memory usage,
# these methods represent the fastest possible solutions for systems of linear equations,
# establishing the theoretical lower bound for both memory and time complexity.

# For extended performance in large-scale computations, consider leveraging GPU-based solvers,
# vectorized C++ libraries, Intel-optimized BLAS routines, and multithreading techniques.



